In [256]:
#reviewText = "We can enter text on any field in Selenium. After entering the text, we may need to remove or clear the text we entered in that field. This interaction with the web elements can be achieved with the help of clear() method."

warnings.filterwarnings('ignore')
from selenium import webdriver
driver = webdriver.Chrome(executable_path= r'C:\\Users\\Wahbeh\\Desktop\\WebAutomation\\chromedriver_win32\\chromedriver.exe')
# For using sleep function because selenium
# works only when the all the elements of the
# page is loaded.
import time
from selenium.webdriver.common.keys import Keys
# Creating an instance webdriver
# browser = webdriver.Chrome()
# browser.get('http://138.197.118.157:8000/segbot/')
# Let's the user see and also load the element
#time.sleep(2)
from selenium.webdriver.common.by import By

def segText(reviewStr):
    browser = webdriver.Chrome()
    browser.get('http://138.197.118.157:8000/segbot/')
    #first_name = browser.find_element(By.ID, "input-firstname")
    textValue = browser.find_element(By.ID, "textfield1")
    #first_name.send_keys("FirstName")
    textValue.send_keys(reviewStr)
    #continue_button = browser.find_element(By.XPATH, value="//input[@value='Continue']")
    submit_btn = browser.find_element(By.XPATH, "/html/body/div/div/div/article/form/div/div[2]/button[1]")
    #continue_button.click()
    submit_btn.click()
    results = browser.find_element(By.ID, "textfield2")
    browser.find_element(By.ID, "textfield1").clear()
    segments = results.text.split("\n")
    return segments

#segText(reviewText)

In [257]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
dictionary_words = set(nltk.corpus.words.words())
from pprint import pprint
from textblob import TextBlob, Word
import spacy
import spacy_legacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models
import itertools 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [258]:
df = pd.read_csv ('sample.csv')
df.head()

,ReviewID,Page Type,Full Text,Rating
0,9093,review,This Apple Watch was a gift for my wife! She l...,5
1,9092,review,Excellent price and works perfectly to tone do...,5
2,9090,review,"It’s an Apple Watch, not much else to say. Gre...",5
3,9089,review,I never thought I would be an Apple Watch guy....,5
4,9088,review,First Apple Watch and I’m enjoying all of the ...,5


In [259]:
df.shapepe

(99, 4)

# Preprocess reviews

In [260]:
def lemmatizeStr(sentence):
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(sentence)
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [261]:
def clean_text(docs):
    for i in range(len(docs)):
        #remove hashtags and mentions
        docs[i] = " ".join(filter(lambda x:x[0]!='#', docs[i].split())) #remove hashtags
        docs[i] = " ".join(filter(lambda x:x[0]!='@', docs[i].split())) #remove mentiones
        
        #Lemmatize text
        docs[i] = lemmatizeStr(docs[i])
       
    
    
        #Remove brands related keywords    
        brands5kw = ["Samsung Galaxy Watch 4 Classic" , "Samsung Galaxy Watch 5 Pro" , "Samsung Galaxy Watch Active 2"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands5kw]
        docs[i] = ' '.join(resultwords)
        
        
        brands4kw = ["Apple watch series 1" , "Apple watch series 2" , "Apple watch series 3" , "Apple watch series 4",
                     "Apple watch series 5" , "Apple watch series 6" , "Apple watch series 7" , "Apple watch series 8" , 
                     "fossil hybrid smartwatch"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands4kw]
        docs[i] = ' '.join(resultwords)

        brands3kw = ["Forerunner 245 Music" , "Forerunner 255 Music" , "Forerunner 645 Music" , 
                     "Forerunner 945 LTE" , "Forerunner 955 Solar" , "Apple watch SE" , "Apple watch Ultra" , 
                     "Fitbit Alta HR" , "Fitbit Aria Air" , "Fitbit Charge 2" , "Fitbit Charge 3" , "Fitbit Charge 4" , 
                     "Fitbit Charge 5" , "Fitbit Charge HR" , "Fitbit Flex 2" , "Fitbit Inspire 2" , 
                     "Fitbit Inspire 3" , "Fitbit Inspire HR" , "Fitbit Sense 2" , "Fitbit Versa 2" , 
                     "Fitbit Versa 3" , "Fitbit Versa 4" , "fitbit versa lite" , "fossil sport smartwatch" , 
                     "Galaxy Watch 2" , "Galaxy Watch 3" , "Galaxy Watch 4" , "Galaxy Watch 5" , 
                     "Galaxy Watch Active" , "garmin captain marvel" , "garmin darth vader" , "garmin Forerunner 10" , 
                     "garmin Forerunner 101" , "garmin Forerunner 110" , "garmin Forerunner 15" , 
                     "garmin Forerunner 201" , "garmin Forerunner 205" , "garmin Forerunner 210" , 
                     "garmin Forerunner 220" , "garmin Forerunner 225" , "garmin Forerunner 230" , 
                     "garmin Forerunner 235" , "garmin Forerunner 245" , "garmin Forerunner 25" , 
                     "garmin Forerunner 255" , "garmin Forerunner 30" , "garmin Forerunner 301" , 
                     "garmin Forerunner 305" , "garmin Forerunner 310XT" , "garmin Forerunner 35" , 
                     "garmin Forerunner 405" , "garmin Forerunner 405CX" , "garmin Forerunner 410" , 
                     "garmin Forerunner 45" , "garmin Forerunner 45S" , "garmin Forerunner 50" , 
                     "garmin Forerunner 55" , "garmin Forerunner 60" , "garmin Forerunner 610" , 
                     "garmin Forerunner 620" , "garmin Forerunner 630" , "garmin Forerunner 645" , 
                     "garmin Forerunner 735XT" , "garmin Forerunner 745" , "garmin Forerunner 910XT" , 
                     "garmin Forerunner 920XT" , "garmin Forerunner 935" , "garmin Forerunner 945" , 
                     "garmin Forerunner 955" , "letsfit fitness racker" , "polar grit x" , "samsung galaxy fit" , 
                     "Samsung Galaxy Gear" , "Samsung Galaxy Watch " , "Samsung Gear 2" , "Samsung Gear Live" , 
                     "Samsung Gear S" , "Samsung Gear S2" , "Samsung Gear S3" , "Samsung Gear Sport" , 
                     "Samsung Watch 2" , "Samsung Watch 3" , "Samsung Watch 4" , "Samsung Watch 5" , 
                     "Samsung Watch Active" , "striiv apex hr" , "striiv dash hr" , "withings move ecg" , 
                     "xiaomi mi band"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands3kw]
        docs[i] = ' '.join(resultwords)
        
        brands2kw = ["Forerunner 10" , "Forerunner 101" , "Forerunner 110" , "Forerunner 15" , "Forerunner 201" , 
                     "Forerunner 205" , "Forerunner 210" , "Forerunner 220" , "Forerunner 225" , 
                     "Forerunner 230" , "Forerunner 235" , "Forerunner 245" , "Forerunner 25" , 
                     "Forerunner 255" , "Forerunner 30" , "Forerunner 301" , "Forerunner 305" , 
                     "Forerunner 310XT" , "Forerunner 35" , "Forerunner 405" , "Forerunner 405CX" , 
                     "Forerunner 410" , "Forerunner 45" , "Forerunner 45S" , "Forerunner 50" , 
                     "Forerunner 55" , "Forerunner 60" , "Forerunner 610" , "Forerunner 620" , 
                     "Forerunner 630" , "Forerunner 645" , "Forerunner 735XT" , "Forerunner 745" , 
                     "Forerunner 910XT" , "Forerunner 920XT" , " Forerunner 935" , "Forerunner 945" , 
                     "Forerunner 955" , "Inspire 2" , "Sense 2" , "amazon halo" , "Aria Air" , "Charge 2" , 
                     "Charge 3" , "Charge 4" , "Charge 5" , "coros apex" , "coros pace" , "Fitbit Ace" , 
                     "Fitbit Alta" , "Fitbit Aria" , "Fitbit Blaze" , "Fitbit Charge" , "Fitbit Flex" , 
                     "Fitbit Flyer" , "Fitbit Force" , "Fitbit Ionic" , "fitbit ionic" , "Fitbit Luxe" , 
                     "Fitbit One" , "Fitbit Sense" , "Fitbit Surge" , "Fitbit Ultra" , "fitbit verca" , 
                     "Fitbit Versa" , "Fitbit Zip" , "Flex 2" , "fossil gen" , "Galaxy Watch" , "garmin approach" , 
                     "garmin fēnix" , "garmin instinct" , "garmin lily" , "garmin luxe" , "garmin swim" , 
                     "garmin venu" , "garmin vívoactive" , "garmin vívofit" , "garmin vívoki" , "garmin vívomove" , 
                     "garmin vívosmart" , "garmin vívosport" , "Gear 2" , "Gear Live" , "Gear S" , "Gear S2" , 
                     "Gear S3" , "Gear Sport" , "huawei band" , "huawei talkband" , "huawei watch" , 
                     "ihealth watch" , "Inspire 3" , "misfit command" , "misfit path" , "misfit ray" , 
                     "misfit vap" , "moov hr" , "moov now" , "mykronoz zefit" , "mykronoz zefit" , "mykronoz zeneo" , 
                     "mykronoz zeround" , "mykronoz zesport" , "mykronoz zetime" , "mykronoz zetrack" , 
                     "mykronoz zewatch" , "polar a360" , "polar a370" , "polar h10" , "polar h9" , "polar ignite" , 
                     "polar m200" , "polar m430" , "polar oh1" , "polar titan" , "polar vantage" , "Samsung Gear" , 
                     "Samsung Watch " , "striiv fusion" , "suunto peak" , "timex ironman" , "Versa 2" , "Versa 3" , 
                     "Versa 4" , "wahoo fitness" , "withings move" , "withings pulds" , "withings scanwatch" , 
                     "withings steel" , "wyze band"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands2kw]
        docs[i] = ' '.join(resultwords)        

        brands1kw = ["Forerunner" , "Forerunner" , "fossil" , "Nüvi " , "Quest"]

        resultwords  = [word for word in docs[i].split() if word.lower() not in brands1kw]
        docs[i] = ' '.join(resultwords)
        
        #Remove feature-related words
        features = ["ability", "quality", "feature", "aspect", "abilities", "qualities", "features", "aspects"]
        resultwords  = [word for word in docs[i].split() if word.lower() not in features]
        docs[i] = ' '.join(resultwords)

    return docs

In [262]:
reviews = np.array(df['Full Text'])
processed_reviews = clean_text(reviews)

In [263]:
#processed_reviews

In [264]:
len(processed_reviews)

99

# Define a function to split each review using SegBot


In [266]:
df["SegmentedReview"] = ""

In [267]:
for i in range(len(processed_reviews)):
    try:
        print(i)
        df.at[i, "SegmentedReview"] =  segText(processed_reviews[i])   
    except Exception:
        print(i)
        pass

0
0
1
1
2
3
4
5
6
7
8
9
9
10
11
11
12
13
14
15
16
17
18
19
19
20
21
22
23
24
24
25
26
26
27
28
29
30
31
32
32
33
34
35
36
37
38
39
40
41
41
42
43
44
45
46
47
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [268]:
df.shape

(99, 5)

In [269]:
df.to_csv('processed.csv')